In [3]:
from flask import Flask,request
import flask
import threading
import os
import sys
import json
import pytz
ist = pytz.timezone('Asia/Kolkata')
from datetime import datetime
import pandas as pd
import numpy as np
# from tensorflow.keras.models import Model, load_model
from calclulate_price import calculate_dynamic_beer_price
import google.generativeai as genai
# from googletrans import Translator
# import googletrans


genai.configure(api_key='')
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)
# loaded_model = load_model('beer_model/beer_price_model_v2.keras')
# scaler = joblib.load('beer_model/beer_price_scaler_v2.pkl')
# feature_names = joblib.load('beer_model/beer_price_feature_names_v2.pkl')




# def predict_single_entry(model, scaler, feature_names, cat_features, num_features, data_dict):
#     """
#     Predict beer price for a single entry.
#     """
#     df = pd.DataFrame([data_dict])
#     df_encoded = pd.get_dummies(df, columns=cat_features, dtype=np.float32)

#     for col in feature_names:
#         if col not in df_encoded.columns:
#             df_encoded[col] = 0

#     # Select only numerical features for scaling
#     X_num = df_encoded[num_features].values.astype(np.float32)
#     X_scaled_num = scaler.transform(X_num)  # Scale numerical features

#     # Combine scaled numerical features and one-hot encoded categorical features
#     X_cat = df_encoded[[col for col in feature_names if col not in num_features]].values.astype(np.float32)
#     X_scaled = np.concatenate([X_scaled_num, X_cat], axis=1)

#     prediction = model.predict(X_scaled, verbose=0).flatten()[0]
#     return prediction

########## FLASK CODE BEGINS ##########

app = Flask(__name__)
# from flask_cors import CORS
# CORS(app)

@app.route('/ping', methods=['GET'])
def ping():
    return flask.Response(response='Successful Ping')

 

@app.route('/invocations', methods=['POST'])
def query_generation():
    start = datetime.now(ist)
    current_timestamp_start = start.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

    beer_data = request.get_json()


    
    try:
        
        # num_features = ['ProductionCost', 'BrandStrength', 'ABV', 'IBU', 'CustomerRatings', 'ShelfLife', 'Temperature', 'Month']
        # cat_features = ['BeerType', 'Packaging', 'Country', 'DemandIndex', 'Import/Local']

        # predicted_price = predict_single_entry(loaded_model, scaler, feature_names, cat_features, num_features, beer_data)
        df = pd.DataFrame([beer_data])
        print(df)
        predicted_price=calculate_dynamic_beer_price(df)
        schema=f""" You are a FAQ chatbot which takes base selling price and predicted price of a beer based on several factors provided in context.
        Use the context provided {beer_data}, the predicted price is {predicted_price} .
        1. You need to provide the reasoning of % price change in base price and predicted price on factors mentioned above.
        2. Use html format strictly
        Create a 4 bullet pointed short and concise answer for above. Use facts only, do not hallucinate.

        Directly provide the answer, do not say i can help with you that, start with heading like Reasoning behind this Price Prediction.
        """

        # messages = [
        #             {"role": "system", "content": schema}
        # ]

        # answer,query_logs = llm(input_sentence,messages)
        
        model=genai.GenerativeModel('gemini-2.0-flash')
        response=model.generate_content(schema)
        # print(response.text)
        reasoning=response.text
        answer=json.dumps({"predicted_price":float(predicted_price),"reasoning":reasoning})
    except Exception as e:
        errors = {'Errors':e}
        print(errors)
        answer="Error"
    
    
    return flask.Response(response=answer, status=200, mimetype='application/json')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port = 8080, threaded = True)

c:\Users\parag\Downloads\beer_price_prediction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
 * Serving Flask app '__main__'


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.6:8080
Press CTRL+C to quit


   BaseSellingPrice UserHistory AgeGroup DeviceType OSType ScreenSize  \
0               5.0     Premium    18-24     Mobile    iOS      Large   

  CompetitorPricing RegionalDemand CustomerRating PaydayProximity  ...  \
0           Similar           High           High            Near  ...   

  ProductMarketingStatus DayOfWeek DemandIndex SpecialOccasion Temperature  \
0                    Low    Friday      Medium            None          22   

   WeatherCondition InventoryLevel CityType WeekdayType  DayType  
0             Sunny           High    Rural     Weekend  Regular  

[1 rows x 21 columns]


127.0.0.1 - - [11/Feb/2025 22:09:31] "POST /invocations HTTP/1.1" 200 -
